<a href="https://colab.research.google.com/github/danbernstein/parkingdirty/blob/master/object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Object Detection API configuration**: In this step, the model for object detection is downloaded, also some copying and deletion of references are done to leave the whole scheme configured.

In [0]:
!git clone https://github.com/tensorflow/models.git
!apt-get -qq install libprotobuf-java protobuf-compiler
!protoc ./models/research/object_detection/protos/string_int_label_map.proto --python_out=.
!cp -R models/research/object_detection/ object_detection/
!rm -rf models

**Imports** needed to run the Object Detection API demonstration

In [0]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt

from PIL import Image
import scipy.misc

from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

import time
import csv
from datetime import datetime

#!pip install shapely

from shapely.geometry import Polygon # to measure overlap between bike lane and bounding boxes


In [0]:
if not os.path.exists('object_detection/input_imgs'):
    os.makedirs('object_detection/input_imgs')
    
    # download parking dirty images here when needed

if not os.path.exists('object_detection/output_imgs'):
    os.makedirs('object_detection/output_imgs')

if not os.path.exists('object_detection/output_csv'):
    os.makedirs('object_detection/output_csv')

In [0]:
# download and read in data
zip_address = 'http://parkingdirty.com/BlockedBikeLaneTrainingSingleCam.zip'

import requests, zipfile, io
r = requests.get(zip_address)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall('object_detection/input_imgs') 

**Configuration** of the model to use, path to the frozen inference graph and extra config elements for the Object detection API implementation.

In [0]:
# What model to download.
# MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'

# model with more accurancy but up to you use a diferent model
MODEL_NAME = 'faster_rcnn_inception_resnet_v2_atrous_coco_2017_11_08'

MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('object_detection/data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90

opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  file_name = os.path.basename(file.name)
  if 'frozen_inference_graph.pb' in file_name:
    tar_file.extract(file, os.getcwd())
    
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')
    
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [0]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [0]:
#mport shutil
#hutil.rmtree('object_detection/output_imgs')
#hutil.rmtree('object_detection/output_csv')

piece of code that represent the concrete detection, calling the TF session

In [0]:
def processimages(path_images_dir, save_directory):

  csv_file = 'object_detection/output_csv/csvfile.csv'

  f = open(csv_file, 'w')
  
  with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
      # configure tf object detection API for boxes, scores, classes, and num of detections
      
      sess.run(tf.global_variables_initializer())
      #img = 1
      # Definite input and output Tensors for detection_graph
      image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
      # Each box represents a part of the image where a particular object was detected.
      detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
      # Each score represent how level of confidence for each of the objects.
      # Score is shown on the result image, together with the class label.
      detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
      detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
      num_detections = detection_graph.get_tensor_by_name('num_detections:0')
      
      for image_path in [os.path.join(path, name) for path, subdirs, files in os.walk(path_images_dir) for name in files]:

        start_time = time.time()
        timestamp = image_path.split(".png")[0]

        try:
            image = Image.open(image_path)
            image_np = load_image_into_numpy_array(image)
        except IOError:
          print("Issue opening "+image_path)
          continue 
              
        # the array based representation of the image will be used later in order to prepare the
        # result image with boxes and labels on it.
        #image_np = load_image_into_numpy_array(image)
        # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
        image_np_expanded = np.expand_dims(image_np, axis=0)
        
        
        # Actual detection.
        (boxes, scores, classes, num) = sess.run(
            [detection_boxes, detection_scores, detection_classes, num_detections],
            feed_dict={image_tensor: image_np_expanded})# Visualization of the results of a detection.

        vis_util.visualize_boxes_and_labels_on_image_array(
            image_np,
            np.squeeze(boxes),
            np.squeeze(classes).astype(np.int32),
            np.squeeze(scores),
            category_index,
            min_score_thresh=0.4,
            use_normalized_coordinates=True,
            line_thickness=8)

        plt.figure(figsize=IMAGE_SIZE)
        #plt.imshow(image_np)

        #plt.imsave(save_directory + image_path, image_np)
        #img += 1

        scores = np.squeeze(scores)
        boxes = np.squeeze(boxes)
        # Return found objects
        #print([category_index.get(i) for i in classes[0]])
        #print(boxes.shape)
        #print(num)
        num_cars_in_bikelane_03, num_cars_in_bikelane_05, num_cars_in_bike_lane_contains, num_bikes_in_bike_lane = 0, 0, 0, 0
        import matplotlib.path as mpltPath

        # the lane polygon is specific to each camera at a particular point in time
        # it could change if the camera's perspective is changed
        # a more robust solution would automatically identify bike lanes
        # lane points identified with: https://www.image-map.net/
        lane = np.array([[158,278],[126,272],[302,115],[310,116]])
        pathbikelane = mpltPath.Path(lane)

  # analyzing the detected objects for which are in the bikelane and converting into a tabular format 

        for i in range(boxes.shape[0]):
           if scores[i] > .4:
              box = tuple(boxes[i].tolist())
              #print(box)

              ymin, xmin, ymax, xmax = box

              # the box is given as a fraction of the distance in each dimension of the image
              # so we have to multiple it by the image dimensions to get the center of each box, relative to the rest of the image
              center_x = (((xmax * 352) - (xmin * 352)) / 2) + (xmin * 352) # x dimension of image
              center_y = (((ymax * 288) - (ymin * 288)) / 2) + (ymin * 288) # y dimension of image
              points = [(center_x, center_y)]
              
              
              obj_area =  ((xmax * 352) - (xmin * 352)) * ((ymax * 288) - (ymin * 288))

              p1 = Polygon([((xmax * 352),(ymax * 288)), ((xmin * 352),(ymax * 288)), ((xmin * 352),(ymin * 288)), ((xmax * 352),(ymin * 288))])
              p2 = Polygon([(158,278),(126,272),(302,115),(310,116)])
              p3 = p1.intersection(p2)
              overlap = p3.area / obj_area
              
              classes_int = np.squeeze(classes).astype(np.int32)

              if classes_int[i] in category_index.keys():
                                  class_name = category_index[classes_int[i]]['name']  

              #print(class_name)
              if class_name in {'car', 'truck', 'bus', 'motorcycle','train','person'}:
                if overlap >= 0.3:
                    num_cars_in_bikelane_03 += 1
                if overlap >= 0.5:
                    num_cars_in_bikelane_05 += 1    
                if pathbikelane.contains_points(points):
                    num_cars_in_bike_lane_contains +=1
              
              if class_name == 'bicycle':
                if pathbikelane.contains_points(points):
                    num_bikes_in_bike_lane += 1      
                    
        #print(num_cars_in_bikelane, num_bikes_in_bike_lane)
    
        # if the image file name contains "not" then assigned 0, otherwise 1, so 1 is blocked, 0 is notblocked
        if os.path.join(PATH_TO_TEST_IMAGES_DIR + "/" + image_path).find('not') is not -1:
          img_labels = 0
        else:
          img_labels = 1 
 
 
        f.write(timestamp + ',' + 
                str(num_cars_in_bikelane_03) + ',' +
                str(num_cars_in_bikelane_05) + ',' + 
                str(num_cars_in_bike_lane_contains) + ',' + 
                str(num_bikes_in_bike_lane) + ',' + 
                str(img_labels) + '\n')

        #print("Process Time " + str(time.time() - start_time))
        scipy.misc.imsave('object_detection/output_imgs/' + os.path.split(image_path)[1], image_np) # save csv to a different directory than annotated images
        
  f.close()
  print('successfully run')
  return csv_file

In [0]:
# DIRECTORY OF IMAGES
#PATH_TO_TEST_IMAGES_DIR = 

# STRUCTURE OF FILE NAME FOR EACH IMAGE
#TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, '2016-09-16 154317 cam135.png'.format(i)) for i in range(1, 3) ]

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

In [15]:
processimages('object_detection/input_imgs', 'object_detection/output_imgs')

TypeError: ignored

In [0]:
# compare computer classified obstacles to manually classified
import pandas
file = pandas.read_csv('object_detection/output_csv/csvfile.csv',
                      names = ['image','obstacle_03','obstacle_contains','obstacle_bikes','label'])
#print(file)
file.obstacle_any = file.obstacle_03 + file.obstacle_contains
pandas.crosstab(file.obstacle_03,file.label)
pandas.crosstab(file.obstacle_contains,file.label)
pandas.crosstab(file.obstacle_any,file.label)

missed = file[file.obstacle_any == 0]
missed.head(30)

In [0]:
#calculate total time of blockage



In [0]:
# read and resize images, also store as X, Y with X as image array and Y as label
#from google.colab import files
#files.download('object_detection/output_csv/csvfile.csv') 